In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from matplotlib import pyplot as plt

In [14]:
#Fill in parameters needed to get k and the function k(T)

In [15]:
#Fill in SS values gathered as found in the memo

In [16]:
#Fill in parameters like tau, ksi, Ccv, etc.

In [13]:
def rates(var):
    #fill in function

SyntaxError: incomplete input (3727379525.py, line 2)

In [ ]:
def flow_valve(x):
    #fill in function

In [10]:
def ManiVar(t):
    return Pso if t<step_time else Pso + delta_Ps

In [ ]:
def diff(t,var,MV):
    #fill in function

In [7]:
def Euler(t,dt,var,MV):
    de_var = diff(t,var,MV)
    var = de_var*dt + var
    return var

In [8]:
def Euler_Integration(tspan,dt,ss_var):
    Values = []
    var = ss_var
    for i,t in enumerate(tspan):
        MV = ManiVar(t)
        var = Euler(t,dt,var,MV)
        Values.append(var)
    return Values